In [1]:
import pandas as pd
import geopandas as gpd
import requests
import lxml.html as lh
from bs4 import BeautifulSoup
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt
import numpy as np
import PIL
import os
import cv2
import glob
from PIL import Image
import matplotlib.ticker as ticker

In [2]:
texas = gpd.read_file("County.shp")
texas = texas.astype({"CNTY_FIPS": int})

In [3]:
def find_max_value(year_range,column_name):
    
    max_value = 0
    
    for i in year: 
    
        I = str(i)
        filename = "TX Production - " + I +".csv"
        production = pd.read_csv(filename)
    
        column = production[column_name]
        temp_max_value = column.max()

        if temp_max_value > max_value:
            max_value = temp_max_value
        else:
            max_value = max_value
        
    return(max_value)

In [4]:
def plot_county_map(merged,year,column_name,max_value):
    
    ax = merged.plot(column=column_name, cmap ='inferno', figsize=(15,9), legend=True, vmin=0, vmax=max_value)
    
    #add title to the map
    ax.set_title(column_name + " Production in " + year, fontdict={'fontsize':15})
    
    #remove axes
    ax.set_axis_off()

In [5]:
def create_movie(year_range,column_name):

    image_frames = []

    max_value = find_max_value(year,column_name)

    for i in year_range: 

        I = str(i)
        filename = "TX Production - " + I +".csv"
        production = pd.read_csv(filename)

        merged = texas.merge(production, left_on = 'CNTY_FIPS', right_on = 'FIPS')

        plot_county_map(merged,I,column_name,max_value)

        #save image
        pic_name = column_name + " - " + I + ".jpg"
        plt.savefig(pic_name)
        
        #add image to image array
        image_frames.append(cv2.imread(pic_name))

        #clear plot and delete image file
        plt.clf()
        os.remove(pic_name)
        
    height,width,layers=image_frames[1].shape

    video=cv2.VideoWriter(column_name + '.mp4',-1,1,(width,height))
    
    for j in range(0,len(year_range)):
        video.write(image_frames[j])
    
    cv2.destroyAllWindows()
    video.release()       